In [1]:
import itertools

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from collections import deque
from functools import reduce
from random import shuffle

from ortools.sat.python import cp_model

### gen data 

In [2]:
def get_all_edge(edges):
    edges_1 = pd.DataFrame(edges, columns=["f", "t"])
    edges_2 = edges_1.copy()
    edges_2.columns = ["t", "f"]

    all_edges = pd.concat([edges_1, edges_2], axis=0, sort=False)
    return all_edges.drop_duplicates()

In [306]:
file = open("./data/gc_1000_5")
file_lines = file.readlines()
file_lines = list(map(lambda s: s.replace("\n", ""), file_lines))
file_lines = list(map(lambda s: s.split(" "), file_lines))
file_lines = pd.DataFrame(file_lines)


data = file_lines.astype("int")
node_count = data.loc[0][0]
edges = data[1:].values
edges_all = get_all_edge(edges)

node_freq = edges_all.groupby("f").size().sort_values(ascending=False)
node_matrix = dict([(f, set(group["t"]))
                    for f, group in edges_all.groupby("f")])
edges_all = list(map(lambda x: frozenset(x), list(edges_all.values)))

print(len(edges))

249482


### gen graph and full connect

In [307]:
G = nx.Graph()
G.add_edges_from(edges)
greedy_result = nx.greedy_color(G)
greedy_result = pd.DataFrame(list(greedy_result.items())).set_index(0)[1].sort_index()
greedy_color_size = len(greedy_result.unique())
print(greedy_color_size)

121


In [309]:
def find_max_full_graph(G):
    max_fc_graph = []
    i = 0
    for graph in nx.find_cliques(G):
        i += 1
        if len(graph) > len(max_fc_graph):
            print(len(graph))
            max_fc_graph = graph
            i = 0
        if i >=1000000:
            break
    return max_fc_graph
max_fc_graph = find_max_full_graph(G)
max_fc_graph_size = len(max_fc_graph)

# fc_graph = list(map(sorted,nx.find_cliques(G)))
# fc_graph = sorted(fc_graph,key=lambda l : - len(l))
# max_fc_graph = fc_graph[0]
# max_fc_graph

9
10
11
12
13


In [310]:
# edges_set = set([frozenset(x) for x in edges])
# select_fc_graph = deque()
# for graph in fc_graph:
#     graph_edges = set([frozenset(x)
#                        for x in itertools.combinations(graph, 2)])
#     common_edges = edges_set & graph_edges
#     if len(common_edges) >= len(graph):
#         edges_set.difference_update(graph_edges)
#         select_fc_graph.append(graph)

# select_fc_graph = list(select_fc_graph)

# print({
#     "edges":len(edges_set),
#     "selece_max_fc_graph":len(select_fc_graph)
# })

### ignore

In [ ]:
# select_fc_nodes

In [ ]:
# def mk_root_full_connect_nodes(node_matrix,root_node):
#     root_node_neighbor = node_matrix[root_node]

#     full_connect_nodes = deque()
#     for two_node in root_node_neighbor:
#         for three_node in  (root_node_neighbor & node_matrix[two_node]):
#             full_connect_nodes.append( frozenset([root_node,two_node,three_node]))
#     return list(set(full_connect_nodes))

# def mk_full_connect_nodes(node_matrix,start_nodes):
#     rt = deque()
#     for i,nodes in enumerate(start_nodes):
#         for new_node in list(reduce(lambda l,r: l & r, map(lambda node:node_matrix[node] ,nodes))):
#             start_nodes[i] = None
#             x = set(nodes)
#             x.add(new_node)
#             rt.append(frozenset(x))
#     return list(set(rt))
# fc_root_node = node_freq.index[0]
# start_fc_nodes = mk_root_full_connect_nodes(node_matrix,fc_root_node)


In [ ]:
# all_fc_nodes = [start_fc_nodes]
# while True:
#     start_nodes = all_fc_nodes[-1]
#     next_nodes = mk_full_connect_nodes(node_matrix,start_nodes)
#     if len(next_nodes) == 0 :
#         break
#     all_fc_nodes.append(next_nodes)
# all_fc_nodes = [(frozenset(y)) for x in all_fc_nodes for y in x  if y is not  None]
# all_fc_nodes = pd.Series(all_fc_nodes).drop_duplicates().values


### model 

In [201]:
xs_nodes = pd.DataFrame(list(range(node_count)),columns=["node"])
xs_nodes["freq"] = xs_nodes.node.map(node_freq) + 1

max_fc_graph_nodes = pd.DataFrame({
    "node":max_fc_graph,
    "lb":range(max_fc_graph_size),
    "ub":range(max_fc_graph_size),
    "value":range(max_fc_graph_size)
})
# max_fc_graph_nodes

In [202]:
xs_nodes = pd.merge(left=xs_nodes, right=max_fc_graph_nodes, how="left")
xs_nodes = xs_nodes.sort_values(["value", "freq"], ascending=False)
xs_nodes.index = range(len(xs_nodes))
xs_nodes["index"] = range(len(xs_nodes))
xs_nodes["lb"] = xs_nodes.lb.fillna(0)
xs_nodes["ub"] = xs_nodes.ub.fillna(xs_nodes["index"])
# xs_nodes["ub"] = xs_nodes.ub.fillna(xs_nodes[["freq","index"]].min(axis=1))
xs_nodes = xs_nodes[["node","lb","ub"]].astype("int").sort_values("node")
xs_nodes


,node,lb,ub
38,0,0,38
26,1,0,26
12,2,0,0
39,3,0,39
11,4,1,1
59,5,0,59
14,6,0,14
21,7,0,21
18,8,0,18
27,9,0,27


In [311]:
model = cp_model.CpModel()
solver = cp_model.CpSolver()

max_color_size = greedy_color_size - 5
xs_node_color = pd.Series([model.NewIntVar(lb=0,ub=min(max_color_size,200),name="n:" + str(i)) for i in range(node_count)])
# xs_node_color = pd.Series(
#     [model.NewIntVar(lb=row["lb"], ub=min(row["ub"], max_color_size), name="node " + str(row["node"]))
#      for index, row in xs_nodes.iterrows()])

for value,node in enumerate(max_fc_graph):
    model.Add(xs_node_color[node] == value)
    

# for edge in edges_set:
#     edge = list(edge)
#     model.Add(xs_node_color[edge[0]] != xs_node_color[edge[1]])

# for nodes in select_fc_graph:
#     model.AddAllDifferent(xs_node_color[nodes].tolist())
# model.AddAllDifferent(xs_node_color[max_fc_graph])

for edge in edges:
    if len(set(edge).difference(max_fc_graph)) == 0:
        continue
    model.Add(xs_node_color[edge[0]] != xs_node_color[edge[1]])


In [312]:
solver.parameters.max_time_in_seconds = 20.0
print(solver.StatusName(solver.Solve(model)),solver.WallTime())
result = xs_node_color.map(lambda x: solver.Value(x))
print(len(result.unique()))
result

UNKNOWN 58.853034300000004


IndexError: list index (0) out of range